In [4]:
# -*- coding: utf-8 -*-
# @author: S. Ramos
import numpy as np
from qutip import *
from scipy import *

In [24]:
"""Qubit repetition
returns list with the number of timess each qubit appears in an instance
"""
def Times(control, instance):
    times = []
    for i in range(control[0]):
        times.append(0)
    for clause in instance:
        for num in clause:
            times[num-1] += 1
    return times

"""Weight generation
Assignes weights to each clause in the instance as a function of the repetition
The weights are then normalized to the same maximum energy
"""
def Weights(control, times, instance):
    coef = []
    for clause in instance:
        coef.append(times[clause[0]-1]+times[clause[1]-1]+times[clause[2]-1]-3)
    norm = sum(coef)/control[1]
    nWeight = [x / norm for x in coef]
    return nWeight

"""Initial Hamiltonian
Creation of the initial product Hamiltonian. Uses the times a qubit appears in the instance
"""
def InitialH(control, sx_list, times):
    HI = 0
    for n in range(control[0]):
        HI += times[n] * 0.5  * (1 - sx_list[n])
    return HI

"""Problem Hamiltonian
Creation of the problem Hamiltonian of the Exact Cover without weights
"""
def ProblemH(control, Z_list, instance):
    HP = 0
    for i in range(control[1]):
        HP += (Z_list[instance[i][0]-1]+Z_list[instance[i][1]-1]+Z_list[instance[i][2]-1]-1)**2.0
    return HP

"""Weighted Hamiltonian
Creation of the problem Hamiltonian of the Exact Cover with weights
"""
def WeightedH(control, Z_list, instance, weight):
    HW = 0
    for i in range(control[1]):
        HW += (weight[i])*(Z_list[instance[i][0]-1]+Z_list[instance[i][1]-1]+Z_list[instance[i][2]-1]-1)**2.00
    return HW


In [25]:
"""Gap evolution of an Exact Cover Problem
Inputs:
    File with information of an Exact Cover Instance
    Time evolution of the problem
    W = True : if a weighted Problem Hamiltonian is desired 
"""
def ExactCoverGap(file, taulist, args, W=True):
    control = list(map(int, file.readline().split()))
    solution = list(map(str, file.readline().split()))
    
    """
    Creation of the operators used for the Hamiltonians using the dimensions of the problem
    """
    si = qeye(2)
    sx = sigmax()
    sz = sigmaz()
    Z = (si - sz)/2.0
    sx_list = []
    Z_list = []
    for n in range(control[0]):
        op_list = []
        for m in range(control[0]):
            op_list.append(si)
        op_list[n] = sx
        sx_list.append(tensor(op_list))
        op_list[n] = Z
        Z_list.append(tensor(op_list))
    
    """
    Reads from the file and separates the clauses of the instance in lists
    """
    instance = []
    for i in range(control[1]):
        instance.append(list(map(int, file.readline().split())))
    
    """
    Creates the initial and problem Hamiltonians, if W == False, the Hamiltonian is not weighted
    """
    times = Times(control, instance)
    H0 = InitialH(control, sx_list, times)
    if W == True:
        weight = Weights(control, times, instance)
        H1 = WeightedH(control, Z_list, instance, weight)
    else:
        H1 = ProblemH(control, Z_list, instance)
        
    """
    Resolves and calculates the gap energy for the given time period. 
    For instances of more than 9 qubits, a sparse method of solving the diagonalization is faster
    """
    Gap = []
    for t in taulist:
        H = (1.0 - (t/args['t_max'])) * H0 + (t/args['t_max']) * H1
        if control[0] > 9:
            evals, ekets = H.eigenstates(sparse=True, eigvals=2, tol=1.0e-3, maxiter=10000)
            Gap.append(evals[1]-evals[0])
        else:
            evals, ekets = H.eigenstates(eigvals=2)
            Gap.append(evals[1]-evals[0])
          
    """
    Checks if the final reult of the evolution coinciedes with the given in the input file
    """
    s = 0
    for i in ekets[0]:
        if i == 0:
            s += 1
        if i != 0:
            ss = s
            break
    form = '{0:0%sb}' % str(control[0])
    if (form.format(ss)) != (''.join(solution)):
        print('error')
    
    return Gap

In [28]:
"""Example
Example of tha calculation of one Gap Energy for an instance of 10 and 14 qubits
"""

taumax = 10.0
taulist = np.linspace(0.0, 10.0, 500)
args = {'t_max': taumax}

instance10 = open('n10i1.txt')
instance14 = open('n14i1.txt')

Gap10 = ExactCoverGap(instance10, taulist, args, W = True)

Gap14 = ExactCoverGap(instance14, taulist, args, W = False)

print('10 qubits:')
print(min(Gap10))
print()
print('14 qubits:')
print(min(Gap14))

10 qubits:
0.212299146269

14 qubits:
0.15505001457
